The first part of this notebook is loading a webpage (loads whole list, so it could be any).  Then moves on to pulling a lot of features from the page.  
### This is the 'test run' for pulling all the features, in notebook 4.

#### First, I need to load the list of web pages to be scraped:

In [1]:
import numpy as np
import pandas as pd
import requests
from time import sleep
from bs4 import BeautifulSoup
import re
from dateutil.parser import *

In [2]:
filename = 'SinglePlayerGames.csv'

df=pd.read_csv(filename, sep=',',header='infer')
df.values

array([['https://steamdb.info/app/493790/'],
       ['https://steamdb.info/app/303720/'],
       ['https://steamdb.info/app/614910/'],
       ..., 
       ['https://steamdb.info/app/665900/'],
       ['https://steamdb.info/app/490390/'],
       ['https://steamdb.info/app/460250/']], dtype=object)

In [3]:
df.head()

,# Links
0,https://steamdb.info/app/493790/
1,https://steamdb.info/app/303720/
2,https://steamdb.info/app/614910/
3,https://steamdb.info/app/392190/
4,https://steamdb.info/app/557330/


In [4]:
Links = []
for line in df.values:
    Links.append(line[0])

In [5]:
len(Links)

5000

In [6]:
Links[0]

'https://steamdb.info/app/493790/'

# Now to pull data from the web!

First, practice on a single page

In [372]:
#url = Links[0]
url = "https://steamdb.info/app/517480/"
response = requests.get(url)

In [373]:
html = response.text

In [374]:
print(html)

<!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="referrer" content="unsafe-url">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0">
<title>VHSoverdose · AppID: 517480 · Steam Database</title>
<link crossorigin="anonymous" rel="stylesheet" href="https://fonts.googleapis.com/css?family=Lato:300,400,400italic,700">
<link rel="stylesheet" href="/static/css/headcrab.52e27913.css">
<link rel="stylesheet" href="/static/css/app.0f98310d.css">
<link rel="publisher" href="https://plus.google.com/107822289875397035811">
<link rel="alternate" type="application/rss+xml" title="SteamDB Blog" href="https://steamdb.info/api/BlogRSS/">
<meta property="twitter:card" content="summary_large_image">
<meta property="twitter:site" content="@SteamDB">
<meta property="og:type" content="website">
<meta property="og:site_name" content="Steam Database">
<meta property="og:title" content="VHSoverdose · AppID: 517480">
<meta property="description" con

In [375]:
soup = BeautifulSoup(html,"lxml")

In [376]:
tables=soup.find_all("table")

## Head Table, basic info

In [377]:
ID_num = soup.find(class_="table table-bordered table-hover table-dark")\
    .findNext("td").findNext("td").text
ID_num

'517480'

In [378]:
applicationCategory = soup.find(itemprop="applicationCategory").text
applicationCategory

'Game'

In [379]:
Name = soup.find(itemprop="name").text
Name

'VHSoverdose'

In [380]:
#Store_Name = soup.find(itemprop="alternateName").text
#Store_Name

In [381]:
Developer = soup.find(itemprop="author").text
Developer

'CallThePolices'

In [382]:
Publisher = soup.find(itemprop="publisher").text
Publisher

'CallThePolices'

In [383]:
Ops = tables[0].find(text = "Supported Systems")
print(tables[0].find(text = "Supported Systems"))

Ops = tables[0].find(text = "Supported Systems").findNext('td')

OS_windows = Ops.find(class_="icon icon-windows tooltipped tooltipped-n")
print(OS_windows)
print(str(OS_windows))
if str(OS_windows) == 'None' : OS_windows = False
else: OS_windows = True
OS_windows

Supported Systems
<i aria-label="Windows" class="icon icon-windows tooltipped tooltipped-n"></i>
<i aria-label="Windows" class="icon icon-windows tooltipped tooltipped-n"></i>


True

In [384]:
OS_mac = Ops.find(class_="icon icon-macos tooltipped tooltipped-n")
print(OS_mac)
print(str(OS_mac))
if str(OS_mac) == 'None' : OS_mac = False
else: OS_mac = True
OS_mac

None
None


False

In [385]:
OS_linux = Ops.find(class_="icon icon-linux tooltipped tooltipped-n")
print(OS_linux)
if str(OS_linux) == 'None' : OS_linux = False
else: OS_linux = True
OS_linux

None


False

In [364]:
SteamPlay = tables[0].find(class_="icon-steamplay")
if str(SteamPlay) == 'None' : SteamPlay = False
else: SteamPlay = True
SteamPlay

False

In [237]:
soup.find(text="Release Date").findNext('td').text

'December 11, 2012 ()'

In [206]:
print(soup.find(text="Release Date").findNext('td').text)
if soup.find(text="Release Date") == None : Release_Date = None
else:
    if soup.find(text="Release Date").findNext('td').text == "Coming Soon": Release_Date = "Coming Soon"
    else:
        Release_Date = soup.find(text="Release Date").findNext('td')
        Release_Date = parse(Release_Date.text.strip("()"))
        Release_Date
Release_Date

Fall 2017


ValueError: Unknown string format

## Front Page info, not in table:
-- Other information from the top of the page, very important stuff!

In [166]:
usr_review_info = soup.find(class_="header-thing header-thing-good tooltipped tooltipped-n")
#pos_usr_review = usr_review_info.findNext().findNext().findNext()
usr_review_info

<span aria-label="96.24% of the 8,287 user reviews for this game are positive.

User Reviews Wilson Score (positive / negative)" class="header-thing header-thing-good tooltipped tooltipped-n" itemprop="aggregateRating" itemscope="" itemtype="http://schema.org/AggregateRating">
<meta content="0" itemprop="worstRating"/>
<meta content="100" itemprop="bestRating"/>
<meta content="95.74" itemprop="ratingValue"/>
<meta content="8287" itemprop="reviewCount"/>
<span class="octicon octicon-thumbsup"></span> 95.74%
<small class="muted"> (
<i class="header-thing-good">8k</i> / 312
)</small>
</span>

In [167]:
worstRating = usr_review_info.find(itemprop="worstRating")
worstRating = str(worstRating).split("\"")[1]
worstRating

'0'

In [168]:
bestRating = usr_review_info.find(itemprop="bestRating")
bestRating = str(bestRating).split("\"")[1]
bestRating

'100'

In [169]:
ratingValue = usr_review_info.find(itemprop="ratingValue")
ratingValue = str(ratingValue).split("\"")[1]
ratingValue

'95.74'

In [170]:
reviewCount = usr_review_info.find(itemprop="reviewCount")
reviewCount = str(reviewCount).split("\"")[1]
reviewCount

'8287'

## Pulling Out the Tables for easier access to the rest

In [246]:
tables=soup.find_all("table")
tables[1]  #the nth table on the page

<table class="table table-fixed table-prices table-hover table-sortable">
<thead>
<tr>
<th class="span3">Currency</th>
<th class="span3 no-sort">Current Price</th>
<th class="span3 sort-default sort-up" colspan="2">Converted Price</th>
<th class="no-sort">Lowest Recorded Price <small class="timeago pull-right" title="2016-02-18T10:01:24+00:00">February 18, 2016</small></th>
</tr>
</thead>
<tbody>
<tr class="owned">
<td class="price-line" data-cc="us" id="js-price-history">
<img alt="us" class="flag flag-us" height="11" src="/static/p.gif" width="16"/> U.S. Dollar
</td>
<td data-sort="0">$9.99</td>
<td class="muted" colspan="2" data-sort="999">Base Price</td>
<td title="December 4, 2014">$0.99 at <span class="price-discount-major">-90%</span></td>
</tr>
<tr>
<td class="price-line" data-cc="uk">
<img alt="uk" class="flag flag-uk" height="11" src="/static/p.gif" width="16"/> British Pound
</td>
<td data-sort="0">£7.99</td>
<td data-sort="1046.3250940633">$10.46</td>
<td class="price-disco

True

### First Sub-table, price info

In [172]:
#current price
Price = tables[1].find(id="js-price-history").findNext('td').text.strip('$')
Price

'11.99'

In [191]:
#lowest historical price, and Maximum sale %
print(tables[1].find(id="js-price-history"))
print(tables[1].find(id="js-price-history")\
    .findNext('td').findNext('td').findNext('td'))
Sale_Price = tables[1].find(id="js-price-history")\
    .findNext('td').findNext('td').findNext('td').text
Sale_Price = Sale_Price.strip('$%')
print(len(Sale_Price.split(" at -")))
print(Sale_Price)
Lowest_Price,Max_Sale = Sale_Price.split(" at -")
Lowest_Price,Max_Sale

<td class="price-line" data-cc="us" id="js-price-history">
<img alt="us" class="flag flag-us" height="11" src="/static/p.gif" width="16"/> U.S. Dollar
</td>
<td title="December 5, 2015">$11.99</td>
1
11.99


ValueError: not enough values to unpack (expected 2, got 1)

### Second Sub-table: Additional Information

In [89]:
genres_raw = tables[2].find(text="Genres").findParent().findNext('td').text
print(genres_raw)
#store_tags = store_tags.split()
#store_tags = store_tags.strip("1234567890:")
genres = genres_raw.split(',')
genres

Action


['Action']

In [96]:
controller_support = tables[2].find(text="controller_support").findParent().findNext('td').text
controller_support

AttributeError: 'NoneType' object has no attribute 'findParent'

In [95]:
tables[2].find(text="controller_support")



#### Metacritic Score. - Professional Review Score

In [31]:
metacritic_score = tables[2].find(text="metacritic_score").findParent().findNext('td').text
metacritic_score

'86'

In [99]:
community_visible_stats = tables[2].find(text="community_visible_stats").findParent().findNext('td').text
community_visible_stats

AttributeError: 'NoneType' object has no attribute 'findParent'

In [33]:
workshop_visible = tables[2].find(text="workshop_visible").findParent().findNext('td').text
workshop_visible

'Yes'

In [35]:
Achievement_Languages_raw = tables[2].find(text="Achievement Languages")\
        .findParent().findNext('td').find_all('i')
Achievement_Languages = []
for tag in Achievement_Languages_raw:
    Achievement_Languages.append(tag.text.strip(":"))
Achievement_Languages

['brazilian',
 'dutch',
 'english',
 'french',
 'german',
 'italian',
 'japanese',
 'koreana',
 'portuguese',
 'russian',
 'spanish',
 'turkish']

In [36]:
languages_num = len(Achievement_Languages)
languages_num

12

In [37]:
community_hub_visible = tables[2].find(text="community_hub_visible").findParent().findNext('td').text
community_hub_visible

'Yes'

In [38]:
store_tags_raw = tables[2].find(text="store_tags").findParent().findNext('td').find_all('a')
#store_tags = store_tags.split()
#store_tags = store_tags.strip("1234567890:")
store_tags = []
for tag in store_tags_raw:
    store_tags.append(tag.text)
store_tags

['Multiplayer',
 'Racing',
 'Soccer',
 'Sports',
 'Competitive',
 'Team-Based',
 'Online Co-Op',
 'Football',
 'Action',
 'Co-op',
 'Fast-Paced',
 'Funny',
 'Great Soundtrack',
 'Local Multiplayer',
 'Split Screen',
 'Local Co-Op',
 'Singleplayer',
 '4 Player Local',
 'Casual',
 'Indie']

In [39]:
releasestate = tables[2].find(text="releasestate").findParent().findNext('td').text
releasestate

'released'

In [40]:
community_hub_visible = tables[2].find(text="community_hub_visible").findParent().findNext('td').text
community_hub_visible

'Yes'

### Third Sub-table: Graphs and Spy
note:  table not present if Table 2 community_visible_stats != 'yes'

Also, this 'table' is actually a list!

In [41]:
#right_now = soup.find(class_="steamspy-stats").findNext()
right_now = soup.find(text=" right now").findPrevious().text
right_now

'43,175'

In [42]:
peak_24_hour = soup.find(text=" 24 hour peak - ").findPrevious().text
peak_24_hour

'53,146'

In [43]:
alltime_peak = soup.find(text=" all-time peak ").findPrevious().text
alltime_peak 

'103,129'

In [44]:
#not available for all
median_playtime_2weeks = soup.find(text=" median playtime in last 2 weeks").findPrevious().text
median_playtime_2weeks

'3.4 hours'

In [45]:
#not availavle for all
average_playtime_2weeks = soup.find(text=" average playtime in last 2 weeks").findPrevious().text
average_playtime_2weeks

'8.8 hours'

### SteamSpy Variables - not available for all games

In [46]:
owners = soup.find(text=" owners").findPrevious().findPrevious().text
owners, owners_unc = owners.split('±')
owners = owners.strip()
owners,owners_unc

('5,731,339', ' 69,145')

In [47]:
players_total = soup.find(text=" players total").findPrevious().findPrevious().text
players_total, players_total_unc = players_total.split('±')
players_total = players_total.strip()
players_total, players_total_unc

('5,499,086', ' 67,749')

In [48]:
owners_played_percent = soup.find(text=" players total").findNext().text
#owners_played_percent = float(owners_played_percent.strip('%'))/100
owners_played_percent.strip('%')
owners_played_percent

'95.95%'

In [49]:
players_2_weeks = soup.find(text=" players in last 2 weeks").findPrevious().findPrevious().text
players_2_weeks, players_2_weeks_unc = players_2_weeks.split('±')
players_2_weeks = players_2_weeks.strip()
players_2_weeks, players_2_weeks_unc

('1,444,392', ' 34,893')

In [50]:
players_2_weeks_percent = soup.find(text=" players in last 2 weeks").findNext().text
#players_2_weeks_percent = float(players_2_weeks_percent.strip('%'))/100
players_2_weeks_percent.strip('%')
players_2_weeks_percent

'25.2%'

In [51]:
#not available for all
median_total_playtime = soup.find(text=" median total playtime").findPrevious().text
median_total_playtime

'19.3 hours'

In [52]:
#not available for all
average_total_playtime = soup.find(text=" average total playtime").findPrevious().text
average_total_playtime

'86.2 hours'

## A few more items from the remaining sub-pages

In [53]:
Packages = soup.find(text = " Packages ").findNext().text
Packages

'20'

In [54]:
DLCs = soup.find(text = " DLCs ").findNext().text
DLCs

'23'

In [55]:
Depots = soup.find(text = " Depots " ).findNext().text
Depots

'19'

In [56]:
4700*1.1/3600

1.4361111111111111